### Test `Unseen Object Clustering (UCN)` Network

In [7]:
"""Test UCN on MuJoCo Scene"""

import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.utils.data

import cv2
import torch.nn as nn
import argparse
import pprint
import numpy as np
import copy
import subprocess



### Run the shell script of `UCN Network`

In [10]:
subprocess.run(["cd ./../../UnseenObjectClustering; ./experiments/scripts/ros_seg_rgbd_add_test_segmentation_realsense.sh $GPU_ID 0"], shell=True)


+ set -e
+ export PYTHONUNBUFFERED=True
+ PYTHONUNBUFFERED=True
+ export CUDA_VISIBLE_DEVICES=0
+ CUDA_VISIBLE_DEVICES=0
+ outdir=./data/checkpoints
+ ./ros/test_images_segmentation.py --gpu 0 --network seg_resnet34_8s_embedding --pretrained ./data/checkpoints/seg_resnet34_8s_embedding_cosine_rgbd_add_sampling_epoch_16.checkpoint.pth --pretrained_crop ./data/checkpoints/seg_resnet34_8s_embedding_cosine_rgbd_add_crop_sampling_epoch_16.checkpoint.pth --cfg experiments/cfgs/seg_resnet34_8s_embedding_cosine_rgbd_add_tabletop.yml
Traceback (most recent call last):
  File "./ros/test_images_segmentation.py", line 13, in <module>
    import tf
ModuleNotFoundError: No module named 'tf'


CompletedProcess(args=['cd ./../../UnseenObjectClustering; ./experiments/scripts/ros_seg_rgbd_add_test_segmentation_realsense.sh $GPU_ID 0'], returncode=1)

In [1]:
import sys
sys.path.append('../')
from model.util_perception import get_cluster_points, list_to_pcd, pcl_to_ros
from model.realsense435 import RealsenseD435i,get_depth_img

camera     = RealsenseD435i(mode='depth')

# 2. Clustering 
subprocess.run(["cd ~/UnseenObjectClustering; ./experiments/scripts/ros_seg_rgbd_add_test_segmentation_realsense.sh $GPU_ID 0"], shell=True)
label_data = np.load('/home/rilab/UnseenObjectClustering/label_data.npy')
depth_data = np.load('/home/rilab/UnseenObjectClustering/depth_data.npy') 
print("Clustering done.")
target_center_point, pc_total_point,cluster_cen_pose_lst = get_cluster_points(depth_pixel=depth_data, label_pixel=label_data, transform_mat=transform_mat)
pc_total_point=list_to_pcd(pc_total_point)
ros_msg = pcl_to_ros(pc_total_point, 'measured/camera_link')
camera.cluster_publishser(ros_msg)
print("Publish cluster points")


ModuleNotFoundError: No module named 'rospy'